# My personal tutorial and toolkit on how to use "Advanced Calculator" universally for string pattern to Y
## installing the Dependencies and stuff

In [ ]:
#install Dependencies on the Environment
!pip install numpy
!pip install pandas
!pip install tensorflow
!pip install torch
!pip install torch-metal #specific for specific hardware
!pip install torchvision
!pip install torchaudio
!pip install gensim
!pip install scikit-learn
!pip install matplotlib


In [ ]:
#Checking the Acceleration specifically for Metal and import torch as ＯＢＯＲ 
import torch as ＯＢＯＲ
ＯＢＯＲ.backends.mps.is_available()

# Load Data

In [11]:
# For now startup_data.csv
import pandas as pd
import os
# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print("Current working directory:", current_directory)
# source https://www.kaggle.com/datasets/manishkc06/startup-success-prediction/data
dataset = pd.read_csv("datasets/EURUSD1440.csv")

TargetColumnValue="volume"
# Read 
print(f"datatype of the dataset var", type(dataset))
dataset.head(10)

Current working directory: /Users/user/Documents/misc/AI/MLBasicsRetraining
datatype of the dataset var <class 'pandas.core.frame.DataFrame'>


,date,open,high,low,close,volume
0,2011.11.02,1.36971,1.38276,1.36357,1.37477,92669
1,2011.11.03,1.37477,1.38541,1.36564,1.38132,97623
2,2011.11.04,1.38132,1.38677,1.37108,1.37898,82606
3,2011.11.07,1.38268,1.38308,1.36808,1.37746,84065
4,2011.11.08,1.37749,1.38467,1.37243,1.38327,79486
5,2011.11.09,1.38327,1.38584,1.35231,1.35416,91607
6,2011.11.10,1.35422,1.36527,1.34835,1.36057,90295
7,2011.11.11,1.36056,1.37947,1.35780,1.37470,73878
8,2011.11.14,1.38090,1.38090,1.35910,1.36321,77676
9,2011.11.15,1.36322,1.36405,1.34961,1.35390,82173


### Identify the target (Y) and the source (X)

Source : Bunch of variable

Target : Will it succeed? with the combination? what is the pattern here? (Status)

# Key Points When Wanting to do Machine Learning
###### sidenote : Yes I know there are a lot of things, I can't keep up with others, so instead I'm just going to assume three steps
1. Read the data, determine the Target
2. Convert all terget into numbers (categorization if below threshold on the unique value (for instance my maximum comfortable unique value is 5) or other stuff) and Clean the data from outliers or null
3. Sequential NN Modeling Prioritization

##### Then lets preprocess the data which is step 2, Convert all target into Numbers and clean the data if there's outliers or null

In [12]:
# Scan for Unique Value on each Column pandas loop

#Automated Data engineering

#We check for the Unique Value to approach whether we want to one_hot_encoding or mapping, lets say if its below <20 lets do Mapping for the column, but this more how comfortable you are with mapping manually and automatically

import numpy as np
#mine is 20 #but it depends on how the data and you like
threshold=10 #Threshold on max when doing manual_traditional_mapping_column before going to toomuchformetohandle_column vector method #can be adapted if its lower than the target_column unique values
hardlimitThreshold=20
ratioUniqueThreshold=0.1
manual_traditional_mapping_column = []
toomuchformetohandle_column = []
id_column = []
## target_column is the final value that we wanted, this will be used later on on defining NN architecture
target_column = TargetColumnValue # Y Axis


#dynamic mapping and vector sum mapping switch based on the target_column
minimumReqthresholdMap = dataset[target_column].nunique()
if (minimumReqthresholdMap <= threshold):
    pass
elif (minimumReqthresholdMap > threshold and minimumReqthresholdMap <= hardlimitThreshold):
    threshold = minimumReqthresholdMap
    print("Adjusting threshold for Mapping")
else:
    print("Training Model may be innacurate due to many lock-on target! Vectorization is going to be use on the target_column dataset")
    pass

print(threshold)

for column in dataset.columns:
    print(f"Type for {column}",dataset[column].dtype)
     # Check if the column contains non-numeric values and does not contain "id" or end with "_id"
      # Count the number of unique non-integer values in the column
        # if ratio unique is more than for instance 0.2 or 0.3 Drop/purge the table to optimize the data for training Accuracy
    unique_values_count = dataset[column].nunique()
    totalDataRow = len(dataset[column])
    ratioUnique = unique_values_count/totalDataRow
    print(f"Column '{column}' has {unique_values_count} unique value(s) with total row {totalDataRow} ratio {ratioUnique} ")

    if ( dataset[column].dtype == "object" ):
        dataset.drop(columns=[column], inplace=True)
        print(f"Column Dropped {column} due to object data {ratioUnique} not string nor int or float")
    if (ratioUnique > ratioUniqueThreshold):
        dataset.drop(columns=[column], inplace=True)
        print(f"Column Dropped {column} due to Erratic Data Row ratio {ratioUnique}")
    else:
        if not np.issubdtype(dataset[column].dtype, np.number) and "id" not in column.lower() and not column.endswith("_id"):
            # Print the column name and the number of unique values        
            if unique_values_count <= threshold:
                # Eligible for manual or traditional mapping
                manual_traditional_mapping_column.append(column)
            else:
                # Too many unique values, recommended for one-hot encoding
                toomuchformetohandle_column.append(column)
        elif "id" in column.lower() or column.endswith("_id"):
            # Append columns containing "id" or ending with "_id" to id_column list
            id_column.append(column)


# So we have two category data to preprocess Narrow Unique Val and Wide Unique Val
print("======================================================")
# For Narrow Unique Val we going to preprocess it with simple Mapping category with integer from 0 to max unique 
# for Wide Unique Val it will have a different handling for preprocessing before feeding into NN
print(f"Narrow Unique Val", manual_traditional_mapping_column)
print(f"Wide Unique Val", toomuchformetohandle_column)
print(f"ID Column", id_column)
print(f"Target", target_column)



Training Model may be innacurate due to many lock-on target! Vectorization is going to be use on the target_column dataset
10
Type for date object
Column 'date' has 3098 unique value(s) with total row 3098 ratio 1.0 
Column Dropped date due to object data 1.0 not string nor int or float


KeyError: "['date'] not found in axis"

#### The start of Data Preprocessing before feeding into the machine

When preparing string data from a pandas DataFrame to feed into a neural network (NN), you typically need to encode the string data into a numerical format, as NNs work with numerical data. Here are some common methods to encode string data for neural network feeding:
## id_column
0. Just filter out non integer or float

## manual_traditional_mapping_column
1. **Label Encoding**:
   - Assigns a unique integer to each category in a categorical variable.
   - Often used for ordinal categorical variables where there is a clear order.
   - Implemented using `LabelEncoder` from scikit-learn or built-in pandas functions.

2. **One-Hot Encoding**:
   - Converts categorical variables into binary vectors.
   - Each category is represented by a binary vector with a 1 in the position corresponding to the category and 0s elsewhere.
   - Used when there is no ordinal relationship among categories.
   - Implemented using `OneHotEncoder` from scikit-learn or `get_dummies()` function in pandas.

## toomuchformetohandle_column

3. **Embedding**:
   - Used for handling high-cardinality/too many category categorical variables.
   - Maps each category to a lower-dimensional dense vector space.
   - Commonly used in natural language processing (NLP) tasks.
   - Implemented using embedding layers in deep learning frameworks like TensorFlow or PyTorch.

4. **Frequency Encoding**:
   - Encodes categories with their frequency of occurrence in the dataset.
   - Useful when categories' frequencies correlate with the target variable.
   - Implemented manually by calculating frequencies and mapping them to categories.

5. **Target Encoding / Mean Encoding**:
   - Replaces categorical values with the mean of the target variable for each category.
   - Used for classification tasks when there's a strong correlation between the categorical variable and the target.
   - Helps capture information about the target variable.
   - Implemented manually or using libraries like category_encoders in Python.

6. **Hashing Encoding**:
   - Converts categorical variables into a fixed-length representation using hashing functions.
   - Reduces memory usage compared to one-hot encoding, especially for high-cardinality variables.
   - Loss of interpretability but can be efficient for large datasets.
   - Implemented using `HashingVectorizer` from scikit-learn or built-in functions in pandas.

In [ ]:
#### Looking from the manual_traditional_mapping_column do an automatic mapping value assignment to the manual manual_traditional_mapping_column 
# Initializing the modified value by starting from pandas dataFrame loaded default dataset
preprocessedDataTrain = dataset

#remove row that contains NaN or missing data
print("removing null/NA and duplicates data")
preprocessedDataTrain.dropna(inplace=True)
preprocessedDataTrain.drop_duplicates(inplace=True)


### Handling id_column

In [ ]:
#for id only allow number on it float64 or integer64
import re #regular expressions time

def clean_numeric(value):
    # Use regular expression to extract numeric parts
    numeric_parts = re.findall(r'\d+\.*\d*', str(value))
    
    # Join numeric parts and convert to float
    cleaned_value = ''.join(numeric_parts)
    
    return float(cleaned_value) if cleaned_value else None


for column in id_column:
    print(preprocessedDataTrain[column].apply(clean_numeric))
    preprocessedDataTrain[column] = preprocessedDataTrain[column].apply(clean_numeric) #.apply seems to apply the whole row with the filter of the function, Sweet!
    
    

preprocessedDataTrain.head(2)

### Handling manual_traditional_mapping_column

In [ ]:

# Then scan for every column that match with the listed manual_traditional_mapping_column array or list idk python, what i care is this runs well :D


#1st handle the easiest mapping manual_traditional_mapping_column

# for each manual_traditional_mapping_column variable scan the preprocessedDataTrain variable (in context the datatype is pandas dataframe) for each column replace the value of the whole column with the count (from 0 to maximum unique value) of self-assigned unique integer for unique value

for column in manual_traditional_mapping_column:
    # Get unique values in the column
    unique_values = preprocessedDataTrain[column].unique()
    
    # Create a dictionary to store unique values as keys and their counts as values
    value_count = {val: idx for idx, val in enumerate(unique_values)}
    
    # Replace values in the column with their counts
    preprocessedDataTrain[column] = preprocessedDataTrain[column].map(value_count)

preprocessedDataTrain.head(5)
## BRUH THERE"S BUILT IN FUNCTION FOR THIS, WHY DO I HAVE TO IMPLEMENT IT MYSELF :MOYAI:


#### Handling toomuchformetohandle_column

In [ ]:
# for each toomuchformetohandle_column variable scan the preprocessedDataTrain variable (in context the datatype is pandas dataframe) for each column that is NOT Integer/float replace the value of the whole column with the count (from 0 to maximum unique value) of self-assigned unique integer for unique value

from gensim.models import Word2Vec
import numpy as np
import pandas as pd


# Do embedding vectorization due to >100 unique thing however if its <=100 then do 
threshold_word2vec=100 # above threshold / >100 is too much cardinality use word2vec // set it to 9999999 to disable

# Iterate through specified columns
for column in toomuchformetohandle_column:
    # Assuming your data is a list of strings
    sentences = preprocessedDataTrain[column].tolist()
    #print(sentences)
    # Tokenize sentences (assuming whitespace tokenization)
    tokenized_sentences = [sentence.split() for sentence in sentences]

    # Train Word2Vec model
    model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)

    # Function to convert a sentence to a vector
    def sentence_to_vector_mean(sentence):
        vector = np.zeros((model.vector_size,))
        count = 0
        for word in sentence:
            if word in model.wv:
                vector += model.wv[word]
                count += 1
        result0 = vector / count if count > 0 else vector
        #print(f"result0 Vector Array {result0}")
        result1 = np.mean(result0)
        #print(f"result1 Vector Array Mean {result1}")
        return result1

    # Convert each sentence to a vector
    vectors = [sentence_to_vector_mean(sentence) for sentence in tokenized_sentences]

    # Replace the column with the vectors
    preprocessedDataTrain[column] = vectors
    
    

#### Final inspection make sure everything already being processed into data appropriately to be feed into the advanced "Calculator" which making sure its not Null or String and everything is number

In [ ]:
preprocessedDataTrain.head(2)

#### Normalization of the Data

In [ ]:
#from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler

standard_scaler = StandardScaler()
min_max_scaler = MinMaxScaler()
#pd.DataFrame(df_scaled_standard, columns=df_encoded.columns)
preprocessedDataTrainTarget = preprocessedDataTrain[target_column]
preprocessedDataTrain = pd.DataFrame(standard_scaler.fit_transform(preprocessedDataTrain), columns=preprocessedDataTrain.columns)
#exclude target_column
preprocessedDataTrain[target_column] = preprocessedDataTrainTarget
#view
preprocessedDataTrain.head(2)

In [ ]:


from sklearn.model_selection import train_test_split
#split the data 69% for training 31% for evaluation


#Convert into target accuracy float training datatype

datatypeProcessing = ＯＢＯＲ.float32 # I don't know what's the difference if this uses float16 or float32, but the default is float32 i presume the Nvidia default


# Assuming your target variable is stored in 'target_column'
X = preprocessedDataTrain.drop(columns=[target_column])
y = preprocessedDataTrain[target_column]

X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.31, random_state=69420) #lets split it 0.31 so we can get the dataset 0.69 or 69%, Nice.

# Optionally, if you want to convert them back to DataFrames
training_data = pd.concat([X_train, y_train], axis=1)
evaluation_data = pd.concat([X_eval, y_eval], axis=1)



### The end of Data Preprocessing

#### The start of NN Architecturing define


## Identify the target of your Data analysis need!

1. **Feedforward Neural Network (FNN)**:
   - This is the simplest form of neural network where connections between nodes do not form a cycle.
   - Information moves in only one direction: forward, from the input nodes, through the hidden layers (if any), and finally to the output nodes.

2. **Convolutional Neural Network (CNN)**:
   - Primarily used for image classification and recognition tasks.
   - CNNs use a variation of multilayer perceptrons designed to require minimal preprocessing.

3. **Recurrent Neural Network (RNN)**:
   - Designed to recognize patterns in sequences of data, such as time series or sentences in natural language processing tasks.
   - RNNs use feedback loops to process sequential data, making them capable of handling inputs of arbitrary length.

4. **Long Short-Term Memory (LSTM)**:
   - A type of RNN architecture that is capable of learning long-term dependencies.
   - LSTMs are well-suited for tasks where the gap between relevant information can be large, such as speech recognition or language modeling.

I know there's more than this, but for now lets focus on these thing


##### Contemplate from the target and the points lets use RNN, since the property of this dataset is "With these combination of circumtances, will his/her endeavour suceed?"

In [ ]:
# we have data from preprocessed training_data and evaluation_data
# now we need to know the dimensions of the input 


#start of the required Model dimension automatic check

def get_model_dimensions(training_data):
    # Extract input size from the number of features (excluding the target column)
    input_size = training_data.shape[1] - 1  # Assuming the target column is the last column

    # Define hidden size based on a rule of thumb or domain knowledge
    hidden_size = 32

    # Define number of layers based on a rule of thumb or domain knowledge
    num_layers = 24

    # Extract output size from the number of unique classes in the target column
    output_size = training_data[target_column].nunique()

    return input_size, hidden_size, num_layers, output_size


input_size, hidden_size, num_layers, output_size = get_model_dimensions(training_data)
print(f"NN Architecture Requirement! : Input size: {input_size}, Hidden size: {hidden_size}, Number of layers: {num_layers}, Output size: {output_size}")
#End of the required model dimensions automatic check


import torch.nn as nn

dynamicArchStubMode=True #disable dynamic Architecturing
## Architecturing Start!
import torch.nn.functional as F

# Architecturing for Transformer (??)



# Architecture for Time Scaling LSTM



# Architecture for RNN Pure NLP Vectorization


# Architecture for Non time scaling FNN only Situation
class ModelArch(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(ModelArch, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # Dense layers
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # Forward propagate through Dense layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        # Softmax activation
        out = F.softmax(x, dim=1)
        return out
# Example usage
# Create an instance of the RNN model
    # Example usage

model = ModelArch(input_size, hidden_size, num_layers, output_size)


# Print the model architecture
print(model)

##### Initialize Acceleration

In [ ]:
# Check if MPS APU is available, or rocm, CUDA GPU is available (wait what's mps and rocm anyway?), otherwise use CPU
if ＯＢＯＲ.cuda.is_available():
    device = ＯＢＯＲ.device('cuda')
    print("Nvidiah exclusive!")
elif ＯＢＯＲ.backends.mps.is_available():
    device = ＯＢＯＲ.device('mps')
    print("Trying to break exclusivity eh?")
elif ＯＢＯＲ.backends.rocm.is_available():
    device = ＯＢＯＲ.device('rocm')
    print("you're a gigachad")
else:
    device = ＯＢＯＲ.device('cpu')
    print("too bad, your training speed will be slow")

model.to(device)

#### Before we get into further training step we're going to learn about loss algorithm, check what is applicable for your specific data
Loss algorithm callbacks, also known as loss functions or objective functions, play a crucial role in training machine learning models. Here are some common types of loss functions used in various machine learning tasks:

1. **Mean Squared Error (MSE)**:
   - Used in regression tasks to measure the average squared difference between the predicted values and the actual values.

2. **Mean Absolute Error (MAE)**:
   - Similar to MSE but measures the average absolute difference between the predicted values and the actual values.

3. **Binary Cross-Entropy Loss**:
   - Used in binary classification tasks to measure the difference between probability distributions, typically when the target variable is binary.

4. **Categorical Cross-Entropy Loss**:
   - Used in multi-class classification tasks to measure the difference between probability distributions when the target variable is categorical.

5. **Sparse Categorical Cross-Entropy Loss**:
   - Similar to categorical cross-entropy loss but used when the target variable is represented as integers instead of one-hot encoded vectors.


### lets do automation
#### TODO: create a code where it check for the target_column variable unique sum if its 2 then use "binary Cross-entropy Loss" but if its more than 2 then "categorical cross-entropy loss"

In [ ]:
# Define loss functions
criterion_mse = nn.MSELoss()
criterion_bce = nn.BCEWithLogitsLoss()
criterion_ce = nn.CrossEntropyLoss()

#check target uniqueness
# uniqueSum = training_data[target_column].nunique()
uniqueSum = training_data[target_column].nunique()
if uniqueSum < 2:
    print("Invalid Data!")
    exit
elif uniqueSum == 2:
    criterion = criterion_bce
elif uniqueSum > 2:
    criterion = criterion_mse

criterion = criterion_ce

In [ ]:
#hyperOS parameter, jk. Hyperparameter
learning_rate = 0.001
num_epochs = 100
batch_size = 32
import torch.optim as optim
#define optimizer. You don't need to learn other optimizer, because others sucks, Adam ftw!
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# define tensor training
    
# Convert training and evaluation data to PyTorch tensors


#main target
X_train_tensor = ＯＢＯＲ.tensor(training_data.drop(columns=[target_column]).values, dtype=datatypeProcessing)
#X_train_tensor = X_train_tensor.unsqueeze(1)
y_train_tensor = ＯＢＯＲ.tensor(training_data[target_column].values, dtype=datatypeProcessing)
#y_train_tensor = y_train_tensor.unsqueeze(1)
# main evaluation
X_eval_tensor = ＯＢＯＲ.tensor(evaluation_data.drop(columns=[target_column]).values, dtype=datatypeProcessing)
X_eval_tensor = X_eval_tensor.unsqueeze(1)
y_eval_tensor = ＯＢＯＲ.tensor(evaluation_data[target_column].values, dtype=datatypeProcessing)
#y_eval_tensor = y_eval_tensor.unsqueeze(1)




In [ ]:
# Convert the training data to DataLoader for batching
import matplotlib.pyplot as plt

train_dataset = ＯＢＯＲ.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = ＯＢＯＲ.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)


train_loss_history = []
train_accuracy_history = []
val_loss_history = []
val_accuracy_history = []


from sklearn.model_selection import KFold

num_folds = 5  # You can adjust the number of folds as needed
kf = KFold(n_splits=num_folds, shuffle=True)

fold = 0
thresholdLoss=0.1 #cut training if its drop below this threshold
thresholdLostEnable=True
for train_index, val_index in kf.split(X_train_tensor):
    fold += 1
    print(f'Fold {fold}/{num_folds}')
    
    # Extract training and validation data for this fold
    X_fold_train, X_fold_val = X_train_tensor[train_index], X_train_tensor[val_index]
    y_fold_train, y_fold_val = y_train_tensor[train_index], y_train_tensor[val_index]
    
    # Convert training data to DataLoader for batching
    fold_train_dataset = ＯＢＯＲ.utils.data.TensorDataset(X_fold_train, y_fold_train)
    fold_train_loader = ＯＢＯＲ.utils.data.DataLoader(dataset=fold_train_dataset, batch_size=batch_size, shuffle=True)

    # Training the model
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        total_loss = 0
        correct = 0
        total = 0
        
        for batch_X, batch_y in fold_train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            # Forward pass
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            _, predicted = ＯＢＯＲ.max(outputs, 1)
            correct += (predicted == batch_y.to(device)).sum().item()
            #print(f"NN Verbose outputs Model Total {outputs} \n NN Verbose outputs y batch tensor {batch_y} \n NN Verbose _ ? {_} \n Activated NN Decoded {predicted}")
            #print(f"varDebug Param {total_loss} {correct} {total}")
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward() #Backpropagation
            optimizer.step()
            total_loss += loss.item()

            total += batch_y.size(0)

        if (loss <= thresholdLoss and thresholdLostEnable):
            print("Preventing Overfitting!")
            break
        # Print average loss and accuracy for each epoch
        accuracy = 100 * correct / total
        loss = total_loss / len(fold_train_loader)
        if ( epoch % 50 == 0 ):
            print(f'Epoch [{epoch + 1}/{num_epochs}], Fold [{fold}/{num_folds}], Loss: {loss}, Accuracy: {accuracy:.2f}%')
        train_accuracy_history.append(accuracy)
        train_loss_history.append(loss)

        # Validation for this Epoch
        model.eval()  # Set the model to evaluation mode
        with ＯＢＯＲ.no_grad():
            outputs = model(X_fold_val.to(device))
            val_loss = criterion(outputs, y_fold_val.to(device))
            _, predicted = ＯＢＯＲ.max(outputs, 1)
            val_correct = (predicted == y_fold_val.to(device)).sum().item()
            val_accuracy = 100 * val_correct / len(y_fold_val)
            if ( epoch % 50 == 0 ):
                print(f'Fold [{fold}/{num_folds}], Validation Loss: {val_loss.item():.4f}, Validation Accuracy: {val_accuracy:.2f}%')
            val_accuracy_history.append(accuracy)
            val_loss_history.append(loss)
        

    

# Save the trained model
ＯＢＯＲ.save(model.state_dict(), 'ＯＢＯＲ_MODEL.pth')

print('Training finished.')


In [ ]:
#shift the plot to increase visibility
plotshift=2

plt.plot(train_loss_history)
plt.plot(range(plotshift, len(val_loss_history) + plotshift), val_loss_history, linestyle='--')  # Shifting validation loss by 4 epochs
plt.title("Model loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Train", "Validation"])
plt.show()

In [ ]:
#shift the plot to increase visibility
plotshift=2

plt.plot(train_accuracy_history)
plt.plot(range(plotshift, len(val_accuracy_history) + plotshift), val_accuracy_history, linestyle='--')  # Shifting validation loss by 4 epochs
#plt.plot(train.history["val_accuracy"])
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Train", "Validation"])
plt.show()

### Evaluate

#### I don't understand this one, yet. but i understand the concept by comparing with the split data of the eval 20%

In [ ]:
# Convert the evaluation data to DataLoader for processing the entire dataset at once
eval_dataset = ＯＢＯＲ.utils.data.TensorDataset(X_eval_tensor, y_eval_tensor)
eval_loader = ＯＢＯＲ.utils.data.DataLoader(dataset=eval_dataset, batch_size=len(eval_dataset))

# Evaluate the model
model.eval()  # Set the model to evaluation mode
total_eval_loss = 0
with ＯＢＯＲ.no_grad():  # Disable gradient calculation
    print(eval_loader)
    for eval_X, eval_y in eval_loader:
        print("spamdebug")
        # Move evaluation tensors to the selected device
        eval_X, eval_y = eval_X.to(device), eval_y.to(device)

        # Forward pass
        eval_outputs = model(eval_X)
        eval_loss = criterion(eval_outputs, eval_y)

        total_eval_loss += eval_loss.item()

# Calculate average evaluation loss
average_eval_loss = total_eval_loss / len(eval_loader)

# You can add other evaluation metrics here such as accuracy, precision, recall, etc.

# Print evaluation results
print(f'Evaluation Loss: {average_eval_loss:.4f}')

# Additional evaluation metrics can be printed here as needed


###